# Handout 05
#### Sara Díaz del Ser

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from termcolor import colored

In [1]:
# Data
ecoli_proteome = './data/ecoli-proteome.faa'
ecoli_orfs_sample = './data/ecoli-orfs-sample.ffn'
ecoli_orfs = './data/ecoli-orfs.ffn'
ecoli_genome = './data/ecoli-genome.fna'
ecoli_genome_sample = './data/ecoli-genome-sample.fna'
ecoli_genes_std= './data/ecoli-genes-standard.ffn'
ecoli_genes = './data/ecoli-genes.ffn'

### Ex. 1 _(6 pts)_ Reading and writing sequences in FASTA format

In previous exercises you were already tasked with reading (at least a single) sequence(s)
from FASTA files. Having a collection of functions available for reading and writing sequence
data is quite handy. There are a wide variety of formats available. However, the FASTA
format is both popular and simple. Thus, it is well suited for writing a set of functions for
dealing with sequences in this format. It does not matter whether the sequence(s) repesented
consist of DNA, RNA or amino acids; however, there might be some conventions on file name
extensions when dealing with one or the other type of sequence. More importantly, a file
may contain a single sequence or more than one sequence and functions for reading and
writing sequences in FASTA format should be able to deal with both cases:

#### (a) Reading multiple sequences from a file

A FASTA file can contain more than one sequence. Write a function ´´´fasta_list(filename)´´´
that reads all sequences from a FASTA file and returns a list of tuples, each tuple con-
taining the header as the first and the sequence as the second element.
Note, that a function written this way would normally first read the complete data
contained in the FASTA file and return all the data in a single data structure, i.e., a
list of tuples.


In [2]:
def fasta_list(filename:str) -> list:
	"""Reads all sequences from fasta file and returns a list of tuples containing of header and sequence"""
	with open(filename, 'r') as f:
		# Read all lines
		all_records = "".join(f.readlines()).split('>')

		# Split into headers and sequences
		fasta_list = [ (record.split('\n',1)[0], record.split('\n',1)[1].replace('\n','')) \
					   for record in all_records if record !='']

		print(colored(f'Successfully read {len(fasta_list)} sequences in {filename}', 'green'))
	return fasta_list

In [ ]:
# Test function
a = fasta_list(ecoli_proteome)

In [5]:
# Test function
b = fasta_list(ecoli_genes)

Successfully read 4321 sequences in the fasta file


####  (b) Using generators to read entries from FASTA files

FASTA files can be very large and calling the function fasta_list could end up using a
lot of memory. However, if you have a FASTA file (e.g. ecoli-genes.ffn) containing a
number of sequences, often you just want to perform some operation for each sequence
separately (maybe something a simple as determine its length). In this case, it is not
really necessary to first store all the elements in memory, rather it is preferable to have
a mechanism that would yield one element (consisting of the header information and
the sequence) at a time without reading all the data first.


In [32]:
def fasta_generator(filename:str):
	"""Reads all sequences from fasta file and returns a list of tuples containing of header and sequence"""
	with open(filename, "r") as f:
		line = f.readline()
		while True:
			if line.startswith('>'):
				header = line.replace('\n','')
				# Read the rest of the lines as long as they're not headers
				seq = ''
				new_line = ''
				while not new_line.startswith('>'):
					seq = seq + str(new_line)
					try:
						new_line = next(f)
					except StopIteration:
						return
					line = new_line
				yield (header, seq.replace('\n',''))

In [33]:
g = fasta_generator(ecoli_genes)


In [34]:
print(next(g))
print(next(g))
print(next(g))

('>gi|556503834|ref|NC_000913.3|:190-255 Escherichia coli str. K-12 substr. MG1655, complete genome', 'ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA')
('>gi|556503834|ref|NC_000913.3|:337-2799 Escherichia coli str. K-12 substr. MG1655, complete genome', 'ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGTATTAGAAGCGCGCGGTCACAACGTTACTGTTATCGATCCGGTCGAAAAACTGCTGGCAGTGGGGCATTACCTCGAATCTACCGTCGATATTGCTGAGTCCACCCGCCGTATTGCGGCAAGCCGCATTCCGGCTGATCACATGGTGCTGATGGCAGGTTTCACCGCCGGTAATGAAAAAGGCGAACTGGTGGTGCTTGGACGCAACGGTTCCGACTACTCTGCTGCGGTGCTGGCTGCCTGTTTACGCGCCGATTGTTGCGAGATTTGGACGGACGTTGACGGGGTCTATACCTGCGACCCGCGTCAGGTGCCCG

In [35]:
# Now attempt to run this code:
name,seq = max(fasta_generator(ecoli_genes),key= lambda x: len(x[1]))
max_length = len(seq)
print("The longest gene is",name,"and contains",max_length,"nucleobases.")

The longest gene is >gi|556503834|ref|NC_000913.3|:2044938-2052014 Escherichia coli str. K-12 substr. MG1655, complete genome and contains 7077 nucleobases.


#### _(c)_ Testing
Test the functions written in (a) and (b) on ecoli-proteome.faa: How many amino
acid sequences are contained in the file? Determine the header and length of the shortest
and longest amino acid sequence of the file.

#### _(d)_ Writing FASTA sequences
Now write a function write_fasta(outfile,header,sequence) that writes the sequence
with header to the opened file outfile in FASTA format. Make sure the the written
header line starts with > and that the sequence is split into lines containing exactly 70
symbols (the final line may contain less than 70 symbols but should not be empty).